In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import copy

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:


#downloading data
ticker = "ETH-USD"
start ='2021-01-01'
end = '2023-12-31'
ohlc_data = yf.download(ticker,start,end,interval='1d')

#variable for storing returns of simple buy and hold strategy
buy_hold = copy.deepcopy(ohlc_data)
buy_hold["Strategy_Return"] = buy_hold["Adj Close"].pct_change()

#Calculating 21-day EMA and 9-day EMA of volume
ohlc_data["EMA"] = ohlc_data["Adj Close"].ewm(span=21,min_periods=21).mean()
ohlc_data["avg_vol"] = ohlc_data["Volume"].ewm(span=9,min_periods=9).mean()
ohlc_data.dropna(how='any',inplace=True)

#function to calculate CAGR
def CAGR(DF):
    df = DF.copy()
    df["cum_return"] = (1+df["Strategy_Return"]).cumprod()
    n = len(df)/365
    cagr = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return cagr


#function to calculate volatility
def volatility(DF):
    df = DF.copy()
    vol = df["Strategy_Return"].std() * np.sqrt(365)
    return vol

#function to calculate max drawdown
def max_dd(DF):
    df = DF.copy()
    df["cum_return"] = (1+df["Strategy_Return"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    return (df["drawdown"]/df["cum_roll_max"]).max()

#function to calculate sharpe ratio
def sharpe(DF,rf):
    df = DF.copy()
    return (CAGR(df)-rf)/volatility(df)

#function to calculate sortino ratio
def sortino(DF,rf):
    df = DF.copy()
    df["Strategy_Return"] = df["Adj Close"].pct_change()
    neg_return = np.where(df["Strategy_Return"]>0,0,df["Strategy_Return"])
    neg_vol = pd.Series(neg_return[neg_return!=0]).std()*np.sqrt(252)
    return (CAGR(df)-rf)/neg_vol
    


initial_capital = 1000
transaction_cost = 0.001  # assuming 0.1% transaction cost per trade
stop_loss_threshold = 0.02 # taking 2% as stoploss limit for our trades

trades = []


# Initialize columns for strategy signals and daily returns
ohlc_data['Position'] = 0  # 1 for long, -1 for short, 0 for neutral

# Implement the strategy
for i in range(1, len(ohlc_data)):
    # Buy signal: Price crosses above EMA with volume > average volume
    if ohlc_data['Close'].iloc[i] > ohlc_data['EMA'].iloc[i] and ohlc_data['Close'].iloc[i-1] <= ohlc_data['EMA'].iloc[i-1] and ohlc_data['Volume'].iloc[i] > ohlc_data['avg_vol'].iloc[i]:
        ohlc_data['Position'].iloc[i] = 1
    # Sell signal: Price drops below EMA
    elif ohlc_data['Close'].iloc[i] < ohlc_data['EMA'].iloc[i] and ohlc_data['Position'].iloc[i-1] == 1:
        ohlc_data['Position'].iloc[i] = 0
    # Short signal: Price crosses below EMA with volume > average volume
    elif ohlc_data['Close'].iloc[i] < ohlc_data['EMA'].iloc[i] and ohlc_data['Close'].iloc[i-1] >= ohlc_data['EMA'].iloc[i-1] and ohlc_data['Volume'].iloc[i] > ohlc_data['avg_vol'].iloc[i]:
        ohlc_data['Position'].iloc[i] = -1
    # Close short position: Price rises above EMA
    elif ohlc_data['Close'].iloc[i] > ohlc_data['EMA'].iloc[i] and ohlc_data['Position'].iloc[i-1] == -1:
        ohlc_data['Position'].iloc[i] = 0
    else:
        ohlc_data['Position'].iloc[i] = ohlc_data['Position'].iloc[i-1]
        
    # Record trades
    if ohlc_data['Position'].iloc[i] != ohlc_data['Position'].iloc[i-1]:
        if ohlc_data['Position'].iloc[i-1] != 0:  # Closing a position
            entry_price = trades[-1]['Entry Price']
            exit_price = ohlc_data['Close'].iloc[i]
            direction = ohlc_data['Position'].iloc[i-1]
            holding_duration = ohlc_data.index[i] - trades[-1]['Entry Date']
            
            trade_return = (exit_price - entry_price) / entry_price * direction
            gross_profit = trade_return * initial_capital
            
            trades[-1]['Exit Price'] = exit_price
            trades[-1]['Gross Profit'] = gross_profit
            trades[-1]['Net Profit'] = gross_profit - (initial_capital * transaction_cost * 2)
            trades[-1]['Holding Duration'] = holding_duration
        if ohlc_data['Position'].iloc[i] != 0:  # Opening a position
            trades.append({
                'Entry Date': ohlc_data.index[i],
                'Entry Price': ohlc_data['Close'].iloc[i],
                'Exit Price': None,
                'Direction': ohlc_data['Position'].iloc[i],
                'Gross Profit': None,
                'Net Profit': None,
                'Holding Duration': None
            })
    
    # Check stop-loss conditions
    if ohlc_data['Position'].iloc[i-1] == 1 and (ohlc_data['Close'].iloc[i] < trades[-1]['Entry Price'] * (1 - stop_loss_threshold)):
        ohlc_data['Position'].iloc[i] = 0
    elif ohlc_data['Position'].iloc[i-1] == -1 and (ohlc_data['Close'].iloc[i] > trades[-1]['Entry Price'] * (1 + stop_loss_threshold)):
        ohlc_data['Position'].iloc[i] = 0
    
        
# Store the cumulative returns in the DataFrame
ohlc_data['Strategy_Return'] = ohlc_data['Position'].shift(1) * ohlc_data['Close'].pct_change()
ohlc_data['Cumulative_Strategy_Return'] = (ohlc_data['Strategy_Return'] + 1).cumprod()

# Convert trades to a DataFrame
trades_df = pd.DataFrame(trades)
trades_df.dropna(how='any',inplace=True)

# Calculate trade statistics
total_closed_trades = len(trades_df)
win_trades = trades_df[trades_df['Net Profit'] > 0]
loss_trades = trades_df[trades_df['Net Profit'] <= 0]

win_rate = len(win_trades) / total_closed_trades if total_closed_trades > 0 else 0
gross_profit = trades_df['Gross Profit'].sum()
gross_loss = trades_df[trades_df['Gross Profit'] < 0]['Gross Profit'].sum()
net_profit = trades_df['Net Profit'].sum()
average_winning_trade = win_trades['Net Profit'].mean() if len(win_trades) > 0 else 0
average_losing_trade = loss_trades['Net Profit'].mean() if len(loss_trades) > 0 else 0
largest_winning_trade = win_trades['Net Profit'].max() if len(win_trades) > 0 else 0
largest_losing_trade = loss_trades['Net Profit'].min() if len(loss_trades) > 0 else 0
average_holding_duration = trades_df['Holding Duration'].mean() if total_closed_trades > 0 else pd.Timedelta(0)




[*********************100%%**********************]  1 of 1 completed


In [4]:
# Print trade statistics
print(f"Total Closed Trades: {total_closed_trades}")
print(f"Win Rate: {win_rate:.2%}")
print(f"Gross Profit: {gross_profit:.2f} USD")
print(f"Gross Loss: {gross_loss:.2f} USD")
print(f"Net Profit: {net_profit:.2f} USD")
print("CAGR of strategy: ",CAGR(ohlc_data))
print(f"Average Winning Trade: {average_winning_trade:.2f} USD")
print(f"Average Losing Trade: {average_losing_trade:.2f} USD")
print(f"Largest Winning Trade: {largest_winning_trade:.2f} USD")
print(f"Largest Losing Trade: {largest_losing_trade:.2f} USD")
print(f"Average Holding Duration: {average_holding_duration}")
print("Maximum drawdown: ",max_dd(ohlc_data))
print("Sharpe ratio: ",sharpe(ohlc_data, 0.025))
print("Sortino ratio: ",sortino(ohlc_data, 0.025))

print("Buy and Hold return of ETH/USD: ",CAGR(buy_hold))
print("Maximum Drawdown of simple Buy & Hold: ",max_dd(buy_hold))
print("Sharpe ratio of simple Buy & Hold: ",sharpe(buy_hold, 0.025))
print("Sortino ratio of simple Buy & Hold: ",sortino(buy_hold, 0.025))

Total Closed Trades: 41
Win Rate: 34.15%
Gross Profit: 1835.63 USD
Gross Loss: -943.14 USD
Net Profit: 1753.63 USD
CAGR of strategy:  0.5425247874420227
Average Winning Trade: 196.48 USD
Average Losing Trade: -36.93 USD
Largest Winning Trade: 689.88 USD
Largest Losing Trade: -95.84 USD
Average Holding Duration: 11 days 14:38:02.926829268
Maximum drawdown:  0.32389971516696786
Sharpe ratio:  1.0487461042741466
Sortino ratio:  0.5165609558202193
Buy and Hold return of ETH/USD:  0.4645802815018034
Maximum Drawdown of simple Buy & Hold:  0.7935123166505244
Sharpe ratio of simple Buy & Hold:  0.5228315900829659
Sortino ratio of simple Buy & Hold:  0.8801209462316589
